In [1]:
import altair as alt
import ast
import base64
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import umap
import uuid

from PIL import Image
from io import BytesIO
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_colwidth', None)

In [2]:
base_name = "2021_Hashimoto_Neural_ODE_and_holographic_QCD_PUB"
project_folder = "diygenomics-projects"
sub_category = "math"
work_bucket = "AdS-CFT"

In [3]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket,
                                       base_name, 'mathpix', *args)

index_col = 'uuid'

if not os.path.exists(file_path('math_embeddings')):
    os.makedirs(file_path('math_embeddings'))

if not os.path.exists(file_path('math_embedding_charts')):
    os.makedirs(file_path('math_embedding_charts'))
    
scaler = StandardScaler()

In [4]:
df = pd.read_csv(file_path('extracted_annotated_math.csv'), index_col=index_col)

In [5]:
df['lstm_character_level_v1_clean_math_embeddings'] = df['lstm_character_level_v1_clean_math_embeddings'].apply(eval)

In [6]:
def scale_image(image):
    width, height = image.size
    aspect_ratio = width / height

    if width > height:
        max_size = int(width / 4)
        max_size = 250 if max_size < 250 else max_size
        new_width = max_size
        new_height = int(max_size / aspect_ratio)
    else:
        max_size = int(height / 4)
        new_height = max_size
        new_width = int(max_size * aspect_ratio)

    resized_image = image.resize((new_width, new_height))

    return resized_image

In [7]:
def format_image(im):
    with BytesIO() as buffer:
        im.save(buffer, 'png')
        data = base64.encodebytes(buffer.getvalue()).decode('utf-8')
    
    return f'data:image/png;base64,{data}'

def load_local_image(image_name):
    img = Image.open(file_path('math_images', image_name))
    img = scale_image(img)
    # img.thumbnail((250, 250))
    return format_image(img)

df['embedded_math_image'] = df['display_math_image'].apply(load_local_image)

In [8]:
def create_or_load_embedding(column):
    math_embeddings_file = file_path('math_embeddings', f'{column}_reduced_embeddings.csv')
    
    if os.path.exists(math_embeddings_file):
        embedding_df = pd.read_csv(math_embeddings_file, index_col=index_col)
    else:
        scaled_data = scaler.fit_transform(df[column].to_list())

        reducer = umap.UMAP(random_state=42)
        embedding = reducer.fit_transform(scaled_data) 

        embedding_df = pd.DataFrame(embedding, columns=['x', 'y'])

        embedding_df['uuid'] = df.index
        embedding_df['paper_annotation'] = df['paper_annotation'].reset_index(drop=True)
        embedding_df['image'] = df['embedded_math_image'].reset_index(drop=True)
        
        embedding_df.set_index('uuid', inplace=True)
        
        embedding_df.to_csv(math_embeddings_file)
    
    return embedding_df

def create_embedding_viz(column):
    embedding_df = create_or_load_embedding(column)
    embedding_df = embedding_df.reset_index(drop=True)

    chart = alt.Chart(embedding_df).mark_circle().encode(
        x='x',
        y='y',
        tooltip=['image']
    ).interactive()
    
    chart = chart.configure_title(
        font='Times New Roman'
    ).configure_axis(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    ).configure_legend(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    ).configure_text(
        font='Times New Roman'
    )

    chart.save(file_path('math_embedding_charts', f'{column}_circle.html'), embed_options={"downloadFileName": column})
    
    chart = alt.Chart(embedding_df).mark_text().encode(
        x='x',
        y='y',
        text='paper_annotation',
        tooltip=['image']
    ).interactive()
    
    chart = chart.configure_title(
        font='Times New Roman'
    ).configure_axis(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    ).configure_legend(
        labelFont='Times New Roman',
        titleFont='Times New Roman'
    ).configure_text(
        font='Times New Roman'
    )

    chart.save(file_path('math_embedding_charts', f'{column}_text.html'), embed_options={"downloadFileName": column})

In [9]:
create_embedding_viz('lstm_character_level_v1_clean_math_embeddings')

In [ ]:
# img = Image.open(file_path('math_images', '1_6998d986-fca2-4650-a4bd-4656e657fa72.png'))
# width, height = img.size
# print(f'{width} {height}')
# new_width = 500  # or whatever value you want
# new_height = int(new_width * height / width)

# max_size = 50
# img.thumbnail((250))

# plt.imshow(resized_image)
# plt.axis('off')  # Remove the axis labels
# plt.show()